In [1]:
import pandas as pd
import folium
from folium import plugins
print(folium.__version__)

0.6.0


In [3]:
# Create URL to JSON file 
url = "https://vownyourdata.zamg.ac.at:9505/api/data/plain"

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='records')

# View the first ten rows
df.head(10)

,alt,auth,eventType,lastUpdate,lat,long,magnitude,magnitudeType,sourceId,time
0,-11.0000,oe,-,2019-03-21T21:44:17.812Z,44.4446,9.9988,3.0,ML,1838207,2019-03-21T21:42:49.771Z
1,-15.0000,ou,-,2019-03-21T20:00:01.264Z,29.8418,163.0871,5.5,mb,1838202,2019-03-21T19:31:19.448Z
2,-10.5918,oadbl,-,2019-03-21T19:00:00.965Z,46.2723,15.4129,1.8,ML,1838197,2019-03-21T18:36:09.955Z
3,-10.0000,EMSC,-,2019-03-21T12:40:55.074Z,27.0500,53.9600,4.6,mb,1838166,2019-03-21T10:03:58.000Z
4,-10.0000,EMSC,-,2019-03-21T13:00:01.392Z,27.0500,53.9600,4.6,mb,1838171,2019-03-21T10:03:58.000Z
5,-10.0000,EMSC,-,2019-03-21T12:40:43.016Z,27.0500,53.9600,4.6,mb,1838161,2019-03-21T10:03:58.000Z
6,-10.0000,QED,-,2019-03-21T12:39:14.229Z,-7.8779,32.0931,5.1,mb,1838151,2019-03-21T09:15:38.350Z
7,-7.0000,EMSC,-,2019-03-21T09:00:00.502Z,38.7600,38.1700,4.1,mw,1837924,2019-03-21T05:51:07.900Z
8,-10.6043,ZAMG:seismomlmb,fe,2019-03-21T12:32:01.190Z,46.7464,12.7544,2.1,ML,1838130,2019-03-21T04:45:53.879Z
9,-6.6805,ZAMG:seismoml,-,2019-03-21T08:00:00.784Z,46.5396,14.7253,0.8,ML,1837907,2019-03-21T02:49:26.208Z


In [4]:
folium_map = folium.Map(location=[48.33, 16.22],
                        zoom_start=4,
                        tiles="CartoDB dark_matter")
for idx, row in df.iterrows():
    color="#E37222" # tangerine
    folium.CircleMarker(location=(row["lat"], row["long"]),
                            radius=row["magnitude"],
                            color=color,
                            popup="{} {}\n{}".format(row["magnitude"], row["magnitudeType"], row["alt"]),
                            fill=True).add_to(folium_map)
folium_map

In [5]:
print(df.count())

alt              876
auth             876
eventType        876
lastUpdate       876
lat              876
long             876
magnitude        876
magnitudeType    876
sourceId         876
time             876
dtype: int64


In [6]:
points = []
times = []
for idx, row in df.iterrows():
    times.append(row['time'])
    points.append({'time':row['time'], 'coordinates':[row['long'], row['lat']], \
                   'magnitude': row['magnitude'],
                   'popup': "{} {} at {}".format(row['magnitude'], row['magnitudeType'], row['alt'])})
    
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'times': [point['time']],
            'popup': point['popup'],
            'icon': 'circle',
            'iconstyle': {
                'fillColor': '#E37222',
                'fillOpacity': 0.5,
                'line': 'false',
                'color': 'red',
                'radius': point['magnitude']*4
            },
        }
    } for point in points
]
m = folium.Map(
    location=[48.33, 16.22],
    tiles='CartoDB dark_matter',
    zoom_start=2,
)

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
    },
    auto_play=True,
    loop=False,
    duration='P1D',
    date_options="YYYY-MM-DD",
    time_slider_drag_update=True
).add_to(m)
m